# Import packages and API Setup

## Import packages

In [2]:
# Data manipulation
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None) # Remove column display limit

# Visualization
import matplotlib.pyplot as plt
import seaborn as sns

# Import HTTP requests library
import requests


## API setup and configuration 

[Alpha Vantage API Documentation](https://www.alphavantage.co/documentation/#)

In [3]:
# Import dotenv_values and load the API key from .env file
from dotenv import dotenv_values 
secrets = dotenv_values("secrets/.env")
alpha_key = secrets["alphavantage_key"]

### Search Endpoint

The Search Endpoint returns the best-matching symbols and market information based on keywords of your choice. The search results also contain match scores that provide you with the full flexibility to develop your own search and filtering logic.  
[Documentation](https://www.alphavantage.co/documentation/#symbolsearch)

In [9]:
# symbol search
def search(keyword):
    # Define base URL with placeholders for symbol and API key
    base_url = "https://www.alphavantage.co/query?function=SYMBOL_SEARCH&keywords={keyword}&apikey={apikey}"

    # Set API key Value
    api_key = alpha_key

    # Replace placeholders with actual values
    url = base_url.format(keyword=keyword, apikey=api_key)

    request = requests.get(url)
    results = request.json()
    df_results = pd.DataFrame(results['bestMatches'])
    return df_results

In [10]:
test_search = search("spy")
test_search

,1. symbol,2. name,3. type,4. region,5. marketOpen,6. marketClose,7. timezone,8. currency,9. matchScore
0,SPY,SPDR S&P 500 ETF Trust,ETF,United States,09:30,16:00,UTC-04,USD,1.0000
1,SPY1.FRK,SSgA SPDR ETFs Europe I Public Limited Company...,ETF,Frankfurt,08:00,20:00,UTC+02,EUR,0.6667
2,SPY2.FRK,SSgA SPDR ETFs Europe I Public Limited Company...,ETF,Frankfurt,08:00,20:00,UTC+02,EUR,0.6667
3,SPY4.FRK,SSgA SPDR S&P 400 US Mid Cap,ETF,Frankfurt,08:00,20:00,UTC+02,EUR,0.6667
4,SPY4.LON,SSgA SPDR ETFs Europe I Public Limited Company...,ETF,United Kingdom,08:00,16:30,UTC+01,USD,0.6667
5,SPY1.DEX,SPDR® S&P 500 Low Volatility UCITS ETF,ETF,XETRA,08:00,20:00,UTC+02,EUR,0.6000
6,SPY2.DEX,SPDR® Dow Jones Global Real Estate UCITS ETF U...,ETF,XETRA,08:00,20:00,UTC+02,EUR,0.6000
7,SPY4.DEX,SSgA SPDR S&P 400 US Mid Cap,ETF,XETRA,08:00,20:00,UTC+02,EUR,0.6000
8,SPY4.PAR,SSgA SPDR S&P 400 US Mid Cap,ETF,Paris,09:00,17:30,UTC+02,EUR,0.6000
9,SPY5.DEX,SPDR® S&P 500 UCITS ETF,ETF,XETRA,08:00,20:00,UTC+02,EUR,0.6000


### Time Series Daily

Alpha Vantage [TIME_SERIES_DAILY API](https://www.alphavantage.co/documentation/#daily)  

This API returns raw (as-traded) daily time series (date, daily open, daily high, daily low, daily close, daily volume) of the global equity specified, covering 20+ years of historical data. The OHLCV data is sometimes called "candles" in finance literature. 

In [11]:
def api_query_daily(symbol, size):
    # Define base URL with placeholders for symbol and API key
    base_url = "https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol={symbol}&outputsize={size}&apikey={apikey}"

    # Set API key Value
    api_key = alpha_key

    # Replace placeholders with actual values
    url = base_url.format(symbol=symbol, size = size, apikey=api_key)
    
    # Send GET request
    request = requests.get(url)
    # Parse the response as JSON
    data = request.json()

    return data

# Load the data

In [12]:
# Query S&P 500 Data
spy_data = api_query_daily("SPY", 'full')

# Convert results to a DataFrame
time_series = spy_data.get("Time Series (Daily)", {})

df_spy = pd.DataFrame.from_dict(time_series, orient='index')
df_spy.index = pd.to_datetime(df_spy.index)

df_spy.head()


,1. open,2. high,3. low,4. close,5. volume
2024-04-24,506.5900,507.3600,503.1350,505.4600,55688584
2024-04-23,501.7800,506.0900,499.5328,505.6500,64633620
2024-04-22,497.8300,502.3800,495.4300,499.7200,67961048
2024-04-19,499.4400,500.4550,493.8600,495.1600,102212587
2024-04-18,501.9800,504.1300,498.5600,499.5200,74548085


In [13]:
df_spy.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 6159 entries, 2024-04-24 to 1999-11-01
Data columns (total 5 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   1. open    6159 non-null   object
 1   2. high    6159 non-null   object
 2   3. low     6159 non-null   object
 3   4. close   6159 non-null   object
 4   5. volume  6159 non-null   object
dtypes: object(5)
memory usage: 288.7+ KB
